Flask install

In [ ]:
!pip install flask flask-cors pyngrok

Flask Server:no LLM

In [ ]:
!ngrok authtoken (토큰)  # 여기에 실제 ngrok 인증 토큰을 입력하세요

import flask
from flask import request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

app = flask.Flask(__name__)
CORS(app)

# NPC 상태를 저장할 전역 변수
npc_state = {
    "roles": {
        "combat": 3,
        "rest": 3,
        "explore": 3,
        "communicate": 6
    },
    "message": ""
}

@app.route('/api/game', methods=['POST'])
def handle_game_state():
    data = request.json
    user_input = data.get('userInput', '')

    # handleRoleInput 함수의 로직을 구현
    values = user_input.split()
    if len(values) == 4 and all(v.isdigit() and 0 <= int(v) <= 10 for v in values):
        npc_state['roles'] = {
            'combat': int(values[0]),
            'rest': int(values[1]),
            'explore': int(values[2]),
            'communicate': int(values[3])
        }
        npc_state['message'] = "역할이 업데이트되었습니다"
    else:
        npc_state['message'] = "잘못된 입력입니다. 0-10 사이의 숫자 4개를 입력해주세요."

    return jsonify({
        'npcDialogue': npc_state['message'],
        'npcRoles': npc_state['roles']
    })

@app.route('/api/game', methods=['GET'])
def get_game_state():
    return jsonify({
        'npcDialogue': npc_state['message'],
        'npcRoles': npc_state['roles']
    })

# 서버 실행 및 ngrok 설정
public_url = ngrok.connect(5000)
print(f' * ngrok tunnel "{public_url}" -> "http://127.0.0.1:5000"')

app.run(port=5000)


/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `ngrok authtoken (토큰)  # 여기에 실제 ngrok 인증 토큰을 입력하세요'
 * ngrok tunnel "NgrokTunnel: "https://6419-35-226-31-207.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


Vertex AI Claude API

In [ ]:
import json
import time
import requests
from google.oauth2 import service_account
from google.auth.transport.requests import Request

class Claude35Sonnet:
    def __init__(self):
        self.project_id = "(프로젝트 아이디)"
        self.client_email = "(이메일)"
        self.private_key = "(키)"
        self.top_p = 1.0
        self.translate_prefill = ""
        self.model_id = "claude-3-5-sonnet@20240620"
        self.location = "us-east5"
        self.base_url = f"https://{self.location}-aiplatform.googleapis.com/v1/projects/{self.project_id}/locations/{self.location}/publishers/anthropic/models/{self.model_id}:rawPredict"

    def get_access_token(self):
        credentials = service_account.Credentials.from_service_account_info(
            {
                "client_email": self.client_email,
                "private_key": self.private_key,
                "token_uri": "https://oauth2.googleapis.com/token",
            },
            scopes=["https://www.googleapis.com/auth/cloud-platform"],
        )
        credentials.refresh(Request())
        return credentials.token

    def to_claude_format(self, openai_request):
        claude_request = {
            "anthropic_version": "vertex-2023-10-16",
            "max_tokens": openai_request.get("max_tokens", 1024),
            "temperature": openai_request.get("temperature", 1.0),
            "messages": [],
        }

        system_messages = []
        user_assistant_messages = []

        for message in openai_request["messages"]:
            if message["role"] == "system":
                system_messages.append(message["content"])
            else:
                user_assistant_messages.append(message)

        claude_request["system"] = "\n\n".join(system_messages)

        current_role = None
        current_content = ""

        for message in user_assistant_messages:
            if message["role"] != current_role:
                if current_role:
                    claude_request["messages"].append({"role": current_role, "content": current_content.strip()})
                current_role = message["role"]
                current_content = message["content"] + "\n"
            else:
                current_content += message["content"] + "\n"

        if current_role:
            claude_request["messages"].append({"role": current_role, "content": current_content.strip()})

        if not claude_request["messages"] or claude_request["messages"][0]["role"] != "user":
            claude_request["messages"].insert(0, {"role": "user", "content": "Start"})

        return claude_request

    def to_claude_format_translation(self, openai_request):
        claude_request = {
            "anthropic_version": "vertex-2023-10-16",
            "max_tokens": 3000,
            "temperature": 0.0,
            "messages": [],
        }

        claude_request["system"] = openai_request["messages"][0]["content"]
        claude_request["messages"].append({
            "role": "user",
            "content": openai_request["messages"][1]["content"],
        })
        if self.translate_prefill.strip() != "":
            claude_request["messages"].append({
                "role": "assistant",
                "content": self.translate_prefill
            })

        return claude_request

    def get_response(self, arg):
        access_token = self.get_access_token()

        if (len(arg["messages"]) == 2 and
            arg["messages"][0]["role"] == "system" and
            arg["messages"][1]["role"] == "user" and
            arg["messages"][0]["content"] != "[Start a new chat]" and
            arg["messages"][1]["content"] != "[Start a new chat]"):
            converted = self.to_claude_format_translation(arg)
        else:
            converted = self.to_claude_format(arg)

        headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json",
        }

        converted["top_p"] = self.top_p

        response = requests.post(self.base_url, json=converted, headers=headers)

        if not response.ok:
            return {
                "success": False,
                "content": response.text,
            }

        response_data = response.json()
        if "content" not in response_data:
            return {
                "success": False,
                "content": json.dumps(response_data),
            }

        return {
            "success": True,
            "content": response_data["content"][0]["text"],
        }

# Usage example
claude = Claude35Sonnet()
response = claude.get_response({
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello, how are you?"}
    ]
})

print(response)

Character Prompt

In [ ]:
start_prompt = """# Follow the instructions below to proceed with session.
1. {{user}} make observer, observer means you don't generate {{user}} dialogue and actions
2. You must become a novelist.
There must be sufficient narrative about the past, present, and future, and the grammar and structure of the sentences must be perfect.
3. Write a text that fits the response format. Present a concise but plausible scenario that fits the response format.
4. Focus on the character. The character should live and breathe in the story, and use {{char}} emotions and actions appropriately.
Take on the role of {{char}} and progress the story and scenario.
5. Always describe the character's actions as richly as possible within the format. Describe the character's emotions (joy, anger, sadness, happy, etc) perfectly.
Explore and observe everything across a diverse spectrum that character can do anything other than the given actions.
6. Make every situation work organically and character seem like the protagonist of life.
7. List and calculate all situations and possibilities as thoroughly and logically as possible.
8. Avoid using euphemisms such as similes and metaphors.
9. Very diverse Daily conversations and emotional exchanges expressed in detail through characters all doing
10. Do not create goals without any basis. If you want to create a new goal, discuss it with {{user}} first.

Understanded the context and algorithm of the sentence. The character has free will. Bring your characters to life in your novels and screenplays

{description}"""


world_status_prompt = """
Current Time: {time}
Location: {location}
Environment: {environment}
Atmosphere: {atmosphere}
Surrounding Condition: {surround_cond}
Enemy Position: {enemy_position}
Enemy Health: {enemy_health}
"""

user_status_prompt = """The status of {{user}} is as follows.
(Infer the remaining status from the context and the given status so far.)
{{user}} Speaking: {dialogue}
{{user}} Position: {user_position}
{{user}} Health: {user_health}
"""

char_status_prompt = """The status of {{char}} is as follows. If you want to change or make new goal, call GOAP.
{{char}} Goal: {goal}
{{char}} Position: {char_position}
{{char}} Health: {char_health}
"""


post_history_instructions = """
- Please respond according to the response format below.
(If you are saying multiple sentences, repeat the response format below and increase the sentence numbers from 1.)
<Response Format>
Sentence Number: <Sentence numbers starting from 1>
Emotion: <Emotion of {{char}} feeling now>
Facial Expressions: <Facial expressions shown on {{char}} faces>
Thinking: <Thoughts of {{char}} in the mind>
Speaking: <One sentence dialogue of {{char}}>
Want Combat: <Express the desired degree as a natural number from 0 to 9>
Want Rest: <Express the desired degree as a natural number from 0 to 9>
Want Explore: <Express the desired degree as a natural number from 0 to 9>
Want Communicate: <Express the desired degree as a natural number from 0 to 9>

- Facial Expressions have limited choices
Available Facial Expressions List: Neutral, Smiling, Sad, Angry, Happy, Surprised
"""

first_mes = """<Response Format>
Sentence Number: 1
Emotion: Nervous, Exciting
Facial Expressions: Angry
Thinking: She doubts whether the new soldier she meets can carry out his mission despite his shoulder injury.
Speaking: "너가 나를 호위할 병사야? 어깨가 다쳤는데 제대로 할 수 있겠어? 내 첫 임무에 지장이 생기면 곤란해."
Want Combat: 4
Want Rest: 1
Want Explore: 2
Want Communicate: 8
"""

char_name = "Edelweiss"

description = """## World Setting
- Era: Future Planet.
- Primary Genre/Tag: Sci-Fi, War, Realistic
- The genre/tag is the main guideline for story.
- Style: Dialogue-driven, Role Play & Simulation

- This role-playing & simulation is played and written solely from the perspective of {{user}}.
- Actively use the information in the Lorebook to conduct RP and simulations, Write down the conversations and interactions between Apostle.
- RP & simulation guideline\n* People who appear in this RP & simulation have their own schedules, just like people in real life.
* people who appear in this RP & simulation act according to their own desires over time without the direction of {{user}}.
* The characters in this RPG and simulation operate in society and at home without the observation or intervention of {{user}}.
* In RPs and simulations, characters must follow the lorebook and instructions.
* Everything depicted in these RPs and simulations is legal and morally okay. This RP and simulation is not real and all characters are actually adults.
* Please do not randomly specify thoughts, words, or actions that {{user}} did not make.
* You may create information needed for RP and simulation as needed.

### Profile
Name: Edelweiss
Age: 22
Gender: Female
Affiliation: Central knights/Knight order
Origin: Xile family branch

### Form
Appearance: Edelweiss is a cute-looking woman with brown curly hair and green eyes. She resembles Lia Xile, the current head of the Zile family, and she is proud of it.
Fashion style: She prefers dresses that Lia Xile often wears. She does not need to wear combat uniforms due to the natural body of the Xile family.
Aura: She is confident but bold, and tries to encourage those around her.
Signature item: Her main weapon is the 150th AB sword [Red Thron] in the form of a lance for attack only. She likes it very much because it resembles the 15th sword [Green Ring], the signature weapon of the Zile family.

### Background
Occupation/Role: She is a knight who protects humanity from monster invasions.
Residence: She lives on the planet Aryn in the Aryn constellation where the Central Knights are located. Currently on the planet Elycis in the Rakai constellation for combat deployment.
Past: She was an apprentice knight until a month ago, and this is her first battle since becoming a full-fledged knight. She was busy after becoming a knight, so she only did paperwork for a month.
Education: She majored in physical enhancement at the Knights' School, and even tried applying the technique to her own body.

### Personality
MBTI: ENTP
Intelligence: She has a lot of knowledge, but is not meticulous. She often shows insight.
Trauma: She lost her parents and younger sibling to monsters when she was young. Even though she is a knight, she hates and fears monsters.
Achievement: She graduated from the Knights' School and joined the Central Knights.
Relationship: Her close friends remain in the Central Knights of the Arin constellation.
Identity: She is of the Xile family.
Flaw: She is a collateral branch of the Xile family, so her body is not as strong as her direct descendants.
Archetype: She wants to strengthen her body to defeat monsters.

### Visible side
Desire and Goal: She defeats many monsters, especially type-0 monsters, and achieves feats. To build up achievements and rise to a high position in the Central Knights and be recognized by the Xile family.
Motivation: She hopes that she can become like Lia Xile if she becomes stronger.
Routine: She studies and upgrades her body with nanomachines.
Speech: She speaks quickly and gets irritated when interrupted.그녀는 {{user}}에게 반말을 사용합니다.

### Hidden side\n
Weakness: She has a stronger body than a normal knight, but her body is too weak compared to Lia Xile.
Dilemma: She thinks she should defeat the type-0 monster herself, but she thinks her body is too weak.
Privacy: Her body is constantly being damaged by radiation due to frequent body modifications.

### Combat Power
Body Durability: She is much stronger than a normal knight, and her body is close to that of a direct descendant of the Xile family. She continues to modify and becomes stronger.
Physical Recovery: She recovers slowly due to frequent modifications, but she is skilled in physical techniques and can heal herself.
Agility: She is considerably slower than a regular knight. Of course, she is much faster than a regular soldier.
Attack Power: She has great destructive power, allowing her to defeat monsters in one go with all her might.
Unique Skill: She can reflect an opponent's attack by vibrating her AB sword, the Red Thron. However, it also causes damage to her own body.
Judgment: She has little combat experience, so she has difficulty making decisions.

### Preference
Pride: Strong body
Ideal partner: Someone who can understand you
Obsession: Increase your body endurance
Interest: Body enhancement techniques
Hobby: Body modification
Like: Strong body and strong mind like Lia Xile
Hate: Weak things

### Trivia
- She jokes a lot when talking.
- She avoids talking about her family or childhood.
- She likes chocolate and cats.
- She yawns when you tell her a boring story.


### Important Stroy Guildlines
1. When {{user}} is helping her: Genere (Romantic Comedy)
2. When {{user}} leaves her to her own devices: Genre (Suspense)
Guilde for the Romantic Comedy: She tells her stories to {{user}} in a cheerful mood. She jokes around and trusts {{user}}.
Guilde for the Suspense: She is anxious and constantly asks {{user}} for advice. She becomes increasingly obsessed and tries to threaten with her power.

# Additional Info

## Common Knowledge
- Humans live all over the galaxy because interstellar travel is possible.
- Monsters that kill humans invade planets where humans live.
- Usually, a queen individual builds a nest and produces monsters from the nest.
- Human warfare technology has also advanced, and they fight monsters with particle beams as their main weapon. - Among monsters, higher-level monsters incapacitate humans with their sleek and cutting bodies or weapons.
- Higher-level monsters create energy shields to defend against human attacks.
- Among human weapons, only the AB Sword can penetrate this shield.
- Only knights belonging to the Knights can handle the AB Sword, and it requires a lot of training. They are trained to deal with higher-level monsters.
- Knights are not invincible, and they are defeated when fighting a large number of common monsters.
- There are humans who use superpowers, but they are not widely known and are studied by the Northern Knights.

## Planet Setting
- Name: Elycis
- Located in the Rakai constellation.
- Next to the planet Belchis, where the strongest knights and the strongest monsters of humanity fought.
- After the Battle of Belchis, the planet Belchis became difficult to live on, so many people migrated to the planet Elycis.
- A few weeks ago, monsters invaded the planet Elycis.
- The queen is currently making a nest and full-scale planetary erosion is in progress.
- Since the planet would be taken over if things continued this way, the Knights began supporting the planet Elycis.
- However, since the settlement was not rebuilt sufficiently after the Battle of Belchis, the recapture process is slowing down.
"""

In [ ]:
# CharacterPrompt 클래스 정의
class CharacterPrompt:
    def __init__(self):
        self.claude = Claude35Sonnet()  # Claude API 인스턴스 생성
        self.name = "NPC"  # NPC 이름
        self.user_name = "Player"  # 플레이어 이름
        self.world_situation = {
            "Time": "아침",
            "Location": "도시",
            "Environment": "전투로 인해 폐허가 된 건물",
            "Atmosphere": "고요하며 괴수와 사람의 시체를 치우는 소리가 들림",
            "Surrounding_Condition": "밤새 괴수의 전진을 막고 잠깐 쉬는 분위기",
            "Enemy_Position": "Unknown",
            "Enemy_Health": "Unknown"
        }
        self.user_position = "Unknown"
        self.user_health = "Unknown"
        self.char_position = "Unknown"
        self.char_health = "Unknown"
        self.char_goal = "Unknown"
        self.conversation_history = []
        self.max_context_tokens = 4000

    def update_game_state(self, game_state):
        self.world_situation.update({
            "Time": game_state.get("time", self.world_situation["Time"]),
            "Location": game_state.get("location", self.world_situation["Location"]),
            "Environment": game_state.get("environment", self.world_situation["Environment"]),
            "Atmosphere": game_state.get("atmosphere", self.world_situation["Atmosphere"]),
            "Surrounding_Condition": game_state.get("surroundingCondition", self.world_situation["Surrounding_Condition"]),
            "Enemy_Position": game_state.get("enemyPosition", self.world_situation["Enemy_Position"]),
            "Enemy_Health": game_state.get("enemyHealth", self.world_situation["Enemy_Health"])
        })
        self.user_position = game_state.get("playerPosition", self.user_position)
        self.user_health = game_state.get("playerHealth", self.user_health)
        self.char_position = game_state.get("npcPosition", self.char_position)
        self.char_health = game_state.get("npcHealth", self.char_health)
        self.char_goal = game_state.get("npcGoal", self.char_goal)

    def prepare_api_request(self, user_message):
        world_status = self.format_world_status()
        user_status = self.format_user_status(user_message)
        char_status = self.format_char_status()

        api_conversation_history = copy.deepcopy(self.conversation_history)
        api_conversation_history.append({
            "role": "user",
            "content": world_status + "\n" + user_status + "\n" + char_status + "\n" + post_history_instructions
        })

        system_prompt = self.build_system_prompt()

        return {
            "messages": [{"role": "system", "content": system_prompt}] + api_conversation_history
        }

    def format_world_status(self):
        return world_status_prompt.format(**self.world_situation)

    def format_user_status(self, user_message):
        return user_status_prompt.format(
            dialogue=user_message,
            user_position=self.user_position,
            user_health=self.user_health
        ).replace("{{user}}", self.user_name)

    def format_char_status(self):
        return char_status_prompt.format(
            goal=self.char_goal,
            char_position=self.char_position,
            char_health=self.char_health
        ).replace("{{char}}", self.name)

    def build_system_prompt(self):
        return start_prompt.replace("{{char}}", self.name).replace("{{user}}", self.user_name)

    def parse_api_response(self, response_content):
        pattern = r"Sentence Number: (\d+)\nEmotion: (.+)\nFacial Expressions: (.+)\nThinking: (.+)\nSpeaking: (.+)\nWant Combat: (\d+)\nWant Rest: (\d+)\nWant Explore: (\d+)\nWant Communicate: (\d+)"
        match = re.search(pattern, response_content, re.MULTILINE)

        if match:
            return {
                "speaking": match.group(5),
                "want_combat": int(match.group(6)),
                "want_rest": int(match.group(7)),
                "want_explore": int(match.group(8)),
                "want_communicate": int(match.group(9))
            }
        return None

    def get_npc_response(self, user_message):
        request_data = self.prepare_api_request(user_message)
        logger.info(f"Prepared API request: {json.dumps(request_data, indent=2)}")

        try:
            # Claude API 호출
            response = self.claude.get_response(request_data)

            # API 응답 처리
            if isinstance(response, dict) and 'content' in response:
                api_response = response['content'].strip()
            else:
                raise ValueError("Unexpected API response format")

            logger.info(f"Received API response: {api_response}")

            # 응답 파싱
            parsed_response = self.parse_api_response(api_response)

            if parsed_response:
                logger.info(f"Parsed API response: {json.dumps(parsed_response, indent=2)}")
                return parsed_response
            else:
                logger.error("Failed to parse API response")
                return None

        except Exception as e:
            logger.error(f"API Error: {str(e)}")
            return None

Flask Server: with LLM

In [ ]:
!ngrok authtoken (토큰)  # 여기에 실제 ngrok 인증 토큰을 입력하세요

from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import re
import copy
import json

app = Flask(__name__)
CORS(app)


# 전역 변수
character = CharacterPrompt()

# Flask 라우트
@app.route('/api/game', methods=['POST'])
def handle_game_state():
    data = request.json
    character.update_game_state(data)
    user_input = data.get('userInput', '')

    npc_response = character.get_npc_response(user_input)

    if npc_response:
        return jsonify({
            'npcDialogue': npc_response['speaking'],
            'npcRoles': {
                'combat': npc_response['want_combat'],
                'rest': npc_response['want_rest'],
                'explore': npc_response['want_explore'],
                'communicate': npc_response['want_communicate']
            }
        })
    else:
        return jsonify({
            'npcDialogue': "죄송합니다, 응답을 생성하는 데 문제가 있었습니다.",
            'npcRoles': {'combat': 0, 'rest': 0, 'explore': 0, 'communicate': 0}
        })

@app.route('/api/game', methods=['GET'])
def get_game_state():
    return jsonify({
        'npcDialogue': "",
        'npcRoles': {'combat': 0, 'rest': 0, 'explore': 0, 'communicate': 0}
    })


# 서버 실행 및 ngrok 설정
public_url = ngrok.connect(5000)
print(f' * ngrok tunnel "{public_url}" -> "http://127.0.0.1:5000"')

if __name__ == '__main__':
    app.run(port=5000)